In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
from nilearn import image as nimg
from nilearn import plotting as nplot
import bids
from nltools.file_reader import onsets_to_dm
from nltools.stats import regress, zscore
from nltools.data import Brain_Data, Design_Matrix
from nltools.stats import find_spikes 
from nilearn.plotting import view_img, glass_brain, plot_stat_map
from bids import BIDSLayout, BIDSValidator

from pathlib import Path

# Making results folder -- change to what this should actually be 
output_dir = Path.cwd() / "results" / "first_level_results"
output_dir.mkdir(exist_ok=True, parents=True)
print(f"Output will be saved to: {output_dir}")

#load in subjects 
#subjects = 

#isolating directory called layout_raw (for events.tsv files) -- will change depending on where data is stored / what computer we're running on
layout_raw = bids.BIDSLayout('/Volumes/Seagate Desktop Drive/KData/', validate=True)


#isolating directory called layout (for fMRIprep derivatives) -- will change depending on where data is stored / what computer we're running on 
layout = bids.BIDSLayout('/Volumes/Seagate Desktop Drive/KData/derivatives', validate=False,
                  config=['bids','derivatives'])
print(layout)

In [ ]:
def load_bids_events(layout,layout_raw, subject, run):
    '''Create a design_matrix instance from BIDS event file'''
    
    tr = layout.get_tr()
    # change lines below -- can change to "mask", change task to "self-other"
    func_files = layout.get(subject=subject,
                        datatype='func', task='selfother',
                        desc='preproc',
                        space='MNI152NLin2009cAsym',
                        extension='nii.gz',
                       return_type='file')
    func_file = nimg.load_img(func_files[run])
    n_tr = func_file.shape[-1]

    onsets = pd.read_csv(layout_raw.get(subject=subject, suffix='events')[run].path, sep='\t')
    # line below is isolating the onset, duration, and trial type columns -- change according to events.tsv format 
    onsets_actual = onsets.iloc[:, [0,1,3]]
    onsets_actual.columns = ['onset', 'duration','trial_type'] # make sure this order matches with what's loaded in as "onsets_actua
    sampling_freq = 1/tr
    n_scans=n_tr
    return onsets_actual, tr, n_scans

In [ ]:
# to iterate through subjects ... 
#for sub in subjects 
sub = '08'
# change lines below -- can change to "mask", change task to "self-other" -- should match the same format as in the load_bids_events function
fmri_imgs = layout.get(subject=sub,
                        datatype='func', task='selfother',
                        desc='preproc',
                        space='MNI152NLin2009cAsym',
                        extension='nii.gz',
                       return_type='file')
# don't need to print when iterating through lots of subjects 
fmri_imgs

In [ ]:
# enumerate over runs to make design matrix + convolve with 6 motion parameters 
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix

# need to define layout, layout raw, sub above for the below code to work 

#set arbitrary initial parameters -- check with Noah/Kelly or Ariel (or Angela?) about these parameters and what makes the most sense 
hrf_model = "spm" #canonical hrf 
high_pass = 0.01 # The cutoff for the drift model is 0.01 Hz.

confound_files = layout.get(subject=sub,
                            datatype='func', task='selfother',
                            desc='confounds',
                           extension="tsv",
                           return_type='file')

# Select confounds -- set right now to just the 6 motion parameters but can add in more/less! 
confound_vars = ['trans_x','trans_y','trans_z',
                 'rot_x','rot_y','rot_z']

final_confounds = confound_vars

design_matrices = []

print("Creating First Level Design matrix ... ")
for idx, img in enumerate(fmri_imgs):
    # Build experimental paradigm
    run = idx
    events,tr,n_scans = load_bids_events(layout,layout_raw, sub, run)
    # Define the sampling times for the design matrix
    frame_times = np.arange(n_scans) * tr
    confound_file = confound_files[run]
    confound_df = pd.read_csv(confound_file, delimiter='\t')
    confound_df = confound_df[final_confounds]
    # Build design matrix with the previously defined parameters
    design_matrix = make_first_level_design_matrix(
        frame_times,
        events,
        hrf_model=hrf_model,
        drift_model="polynomial",
        drift_order=3,
        add_regs=confound_df,
        add_reg_names=confound_vars,
        high_pass=high_pass,
    )
    design_matrix = design_matrix.iloc[:,0:11] # taking out constant intercept and adding in an intercept for each individual run
    # this allows average voxel to vary across runs instead of assuming its constant 
    if idx == 0:
        design_matrix['intercept1'] = 1 
        design_matrix['intercept2'] = 0
        design_matrix['intercept3'] = 0
    else if idx == 1:
        design_matrix['intercept1'] = 0
        design_matrix['intercept2'] = 1 
        design_matrix['intercept3'] = 0
    else: 
        design_matrix['intercept1'] = 0
        design_matrix['intercept2'] = 0 
        design_matrix['intercept3'] = 1
    # put the design matrices in a list
    design_matrices.append(design_matrix)
    
# can visualize the design matrix with the line below 
#plot_design_matrix(design_matrices[1])
print("First Level Design Matrix completed")


run = 0
# Iterate on contrasts
for run in design_matrices:
    plot_design_matrix(run)

In [ ]:
from nilearn.glm.first_level import FirstLevelModel
from nilearn import plotting
from nilearn.plotting import plot_contrast_matrix

# Creating contrasts -- 
### change depending on task / ideal contrasts 
contrast_matrix = np.eye(design_matrix.shape[1])
basic_contrasts = {
    column: contrast_matrix[i]
    for i, column in enumerate(design_matrix.columns)
}

contrasts = {
    "self-other": (basic_contrasts["self"] - basic_contrasts["other"]),
    "self-fix": (basic_contrasts["self"] - basic_contrasts["fix"]),
    "other-fix": (basic_contrasts["other"] - basic_contrasts["fix"]),
    "case-fix": (basic_contrasts["case"] - basic_contrasts["fix"]),

}

from nilearn.plotting import plot_contrast_matrix
for key, values in contrasts.items():
    plot_contrast_matrix(values, design_matrix=design_matrices[0])
    plt.suptitle(key)
plt.show()

from nilearn.glm.first_level import FirstLevelModel

print("Fitting first-level GLM ...")
fmri_glm = FirstLevelModel()
fmri_glm = fmri_glm.fit(fmri_imgs, design_matrices=design_matrices)

print("First-level completed!")

In [ ]:
# plotting: 

from nilearn import plotting
# creating mean img for plotting purposes 
from nilearn.image import mean_img

mean_image = mean_img(fmri_imgs[0])

print("Computing contrasts")

# Iterate on contrasts
for contrast_id, contrast_val in contrasts.items():
    print(f"\tcontrast id: {contrast_id}")
    # compute the contrasts
    z_map = fmri_glm.compute_contrast(contrast_val, output_type="z_score")
    # plot the contrasts as soon as they're generated
    # the display is overlaid on the mean fMRI image
    # a threshold of 3.0 is used, more sophisticated choices are possible
    plotting.plot_stat_map(
        z_map,
        bg_img=mean_image,
        threshold=3,
        display_mode="z",
        cut_coords=3,
        black_bg=True,
        title=contrast_id,
    )
    plotting.show()

In [ ]:
# Interactive plotting 
view = plotting.view_img(z_map, threshold=3)
# In a Jupyter notebook, if ``view`` is the output of a cell, it will
# be displayed below the cell
view